In [197]:
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import warnings,os,datetime,time

import pandas as pd

from nltk import sent_tokenize,word_tokenize

import pickle

In [2]:
basedir = '..//..//..//TF_data//BLOGENTRIES//blogs//'

files = os.listdir(basedir)[:250]

df = pd.DataFrame()
for c_file in files:
    
    try:
        with open(basedir+c_file) as fp: soup = BeautifulSoup(fp, 'xml')
    except:
        continue
    cells = soup.findAll(['date','post'])

    data = []
    for k in range(0,len(cells),2):
        data.append([
            str(cells[k]).replace('<date>','').replace('</date>','').replace(',','-'),
            str(cells[k+1]).replace('<post>','').replace('</post>','').replace('\n','').replace('\t','').lstrip().rstrip()
        ])
    cf = pd.DataFrame(data).rename(columns={0:'Date Post',1:'Text Post'})
    c_ID,c_Gender,c_Age,c_Topic,c_AstroSign = c_file.split('.')[:-1]

    cf['ID Poster'] = c_ID
    cf['Gender Poster'] = c_Gender
    cf['Age Poster'] = c_Age
    cf['Topic Channel'] = c_Topic
    cf['Astro Sign Poster'] = c_AstroSign
    
    df = pd.concat([df,cf]).reset_index(drop=True)

In [ ]:
df0 = df[['Text Post','ID Poster']]

In [ ]:
df0 = df0[df0['ID Poster'].isin(df0['ID Poster'].value_counts().sort_values(ascending=False).index[:4])]
df0['Text Sentence'] = df0['Text Post'].apply(lambda x: [xxx for xx in sent_tokenize(x) for xxx in xx.split(',')])
df0 = df0.drop(columns=['Text Post'])

In [ ]:
df0 = df0.explode(column='Text Sentence')

In [ ]:
df0 = df0[df0['Text Sentence'].apply(lambda x: type(x)==str)].reset_index(drop=True)
df0['Text Sentence'] = df0['Text Sentence'].apply(lambda x: x.lower())
df0['Length Sentence'] = df0['Text Sentence'].apply(lambda x: len(word_tokenize(x)))

In [180]:
df0 = df0[df0['Length Sentence']>=20]
df0 = df0.sample(frac=1).reset_index(drop=True)
df0 = df0.drop(columns=['Length Sentence'])

In [198]:
df0.to_pickle('.\\DATA\\test_dataset_normalized.p')